In [2]:
import tensorflow as tf
import pickle
from PIL import Image
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2
from sklearn.utils.class_weight import compute_class_weight

In [3]:
#hyper parameters:
BATCH_SIZE = 32
NUM_EPOCHS = 20

### Model Architecture

In [ ]:
# Pretrained model for finetunning. This results in saved weights that are too large for github.
def ResNetPredictor(num_classes):
    base_model = tf.keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

In [33]:
class CardPredictor(tf.keras.Model):
    def __init__(self, num_of_classes):
        super(CardPredictor, self).__init__()
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                                tf.keras.layers.RandomContrast(0.1),
                                tf.keras.layers.GaussianNoise(0.1),
                                tf.keras.layers.RandomBrightness(0.1),
                                tf.keras.layers.RandomTranslation(0.1, 0.1),
                            ])
        
        self.architecture = [        
                tf.keras.layers.InputLayer((300, 300, 3)),
                #data_augmentation,

                tf.keras.layers.Conv2D(64, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(128, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(256, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(512, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(1024, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.GlobalAveragePooling2D(),                                
                
                tf.keras.layers.Dense(256, activation='relu'),               # Fully connected layer
                tf.keras.layers.Dropout(0.5),                                # Prevent overfitting
                tf.keras.layers.Dense(num_of_classes, activation='softmax')
                ]
        
        
        self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor_classes_"+ str(num_of_classes))
        
    def call(self, x):
        """ Passes input image through the network. """
        return self.sequential(x)

### Dataset Loading

In [5]:
#TRAIN
with open('train.pkl', 'rb') as file:
    train_data = pickle.load(file)
    
train_suit_data = {}
train_rank_data = {}

for key, inner_dict in train_data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['concept_label']
    rank = value[:13]
    suit = value[13:]
    train_rank_data[img_path] = rank
    train_suit_data[img_path] = suit
    
with open('val.pkl', 'rb') as file:
    test_data = pickle.load(file)

#TEST
test_suit_data = {}
test_rank_data = {}

for key, inner_dict in test_data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['concept_label']
    rank = value[:13]
    suit = value[13:]
    test_rank_data[img_path] = rank
    test_suit_data[img_path] = suit

In [18]:
# Define the image loading and preprocessing function
def load_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Preprocess the dataset for training
train_rank_dataset = (
    tf.data.Dataset.from_tensor_slices((list(train_rank_data.keys()), list(train_rank_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
train_suit_dataset = (
    tf.data.Dataset.from_tensor_slices((list(train_suit_data.keys()), list(train_suit_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
test_rank_dataset = (
    tf.data.Dataset.from_tensor_slices((list(test_rank_data.keys()), list(test_rank_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
test_suit_dataset = (
    tf.data.Dataset.from_tensor_slices((list(test_suit_data.keys()), list(test_suit_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

half_size = int(len(test_rank_dataset)*0.5)
val_rank_dataset = test_rank_dataset.take(half_size)
test_rank_dataset = test_rank_dataset.skip(half_size)
val_suit_dataset = test_suit_dataset.take(half_size)
test_suit_dataset = test_suit_dataset.skip(half_size)

print("Train dataset size:", len(train_rank_dataset))
print("Validation dataset size:", len(val_rank_dataset))
print("Test dataset size:", len(test_rank_dataset))

Train dataset size: 219
Validation dataset size: 47
Test dataset size: 48


### Model Initialization

In [37]:
suit_model = CardPredictor(4)
suit_model.build((None, 300, 300, 3))
suit_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
rank_model = CardPredictor(13)
rank_model.build((None, 300, 300, 3))
rank_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

### Model Training

In [11]:
suit_model.fit(train_suit_dataset, 
                validation_data=val_suit_dataset, 
                epochs=NUM_EPOCHS, 
                verbose=1,
                callbacks=[
                    tf.keras.callbacks.ModelCheckpoint(
                        filepath='suit_model_ckpt.keras',
                        save_best_only=True,
                    )
                ])

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 153s 653ms/step - accuracy: 0.2516 - loss: 1.4309 - val_accuracy: 0.2500 - val_loss: 1.3877
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 157s 667ms/step - accuracy: 0.2533 - loss: 1.3868 - val_accuracy: 0.2473 - val_loss: 1.3868
Epoch 3/20
  1/219 ━━━━━━━━━━━━━━━━━━━━ 44:29 12s/step - accuracy: 0.3125 - loss: 1.3908

KeyboardInterrupt: 

In [38]:
rank_model.fit(train_rank_dataset, 
                validation_data=val_rank_dataset, 
                epochs=NUM_EPOCHS, 
                verbose=1,
                callbacks=[
                    tf.keras.callbacks.ModelCheckpoint(
                        filepath='rank_model_ckpt.keras',
                        save_best_only=True,
                    )
                ])

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 111s 457ms/step - accuracy: 0.1444 - loss: 2.3183 - val_accuracy: 0.2294 - val_loss: 2.0922
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 110s 462ms/step - accuracy: 0.2170 - loss: 2.0698 - val_accuracy: 0.2879 - val_loss: 1.8860
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 113s 476ms/step - accuracy: 0.3372 - loss: 1.7376 - val_accuracy: 0.3251 - val_loss: 1.8034
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 110s 455ms/step - accuracy: 0.3850 - loss: 1.7572 - val_accuracy: 0.4555 - val_loss: 2.0006
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 116s 488ms/step - accuracy: 0.4414 - loss: 1.8803 - val_accuracy: 0.5459 - val_loss: 2.3478
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 112s 468ms/step - accuracy: 0.5936 - loss: 1.4548 - val_accuracy: 0.6775 - val_loss: 1.1679
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 111s 464ms/step - accuracy: 0.6676 - loss: 1.4233 - val_accuracy: 0.6144 - val_loss: 2.3663
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 110s 463ms/step - accuracy: 0.6737 -

KeyboardInterrupt: 

### Model Load From CheckPoint

In [31]:
suit_model.load_weights("suit_model_ckpt.keras")

In [39]:
rank_model.load_weights("rank_model_ckpt.keras")

### Model Evaluation

In [34]:
suit_model.evaluate(test_suit_dataset, verbose=1)

48/48 ━━━━━━━━━━━━━━━━━━━━ 9s 107ms/step - accuracy: 0.9980 - loss: 0.0409


[0.09357084333896637, 0.9966931343078613]

In [40]:
rank_model.evaluate(test_rank_dataset, verbose=1)

48/48 ━━━━━━━━━━━━━━━━━━━━ 9s 113ms/step - accuracy: 0.6823 - loss: 1.3692


[1.3898173570632935, 0.6673280596733093]